### 1. Напишите код для буквального слушающего и для прагматического слушающего.

<font color="green"> Дополненный код: </font>

In [7]:
import numpy as np
import pandas as pd


class RSA:
    """Implementation of the core Rational Speech Acts model.

    Parameters
    ----------
    lexicon : `np.array` or `pd.DataFrame`
        Messages along the rows, states along the columns.
    prior : array-like
        Same length as the number of colums in `lexicon`.
    costs : array-like
        Same length as the number of rows in `lexicon`.
    alpha : float
        Default: 1.0
    """
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        """Literal listener predictions, which corresponds intuitively
        to truth conditions with priors.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.

        """
        return rownorm(self.lexicon * self.prior) # Возвращаем матрицу self.lexicon с учетом self.prior

    def speaker(self):
        """Returns a matrix of pragmatic speaker predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to states, the columns to states.
        """
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        """Returns a matrix of pragmatic listener predictions.

        Returns
        -------
        np.array or pd.DataFrame, depending on `self.lexicon`.
        The rows correspond to messages, the columns to states.
        """
        # Ваш код тут
        speaker_probs = self.speaker().T
        weighted_speaker_probs = speaker_probs * self.prior    # вычисляем ожидаемые значения для каждого сообщения для каждого сообщения, учитывая lexicon, prior, costs и alpha
        return rownorm(weighted_speaker_probs) # Ваш код тут


def rownorm(mat):    # для нормализации строк матрицы
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['hat', 'glasses']
    states = ['r1', 'r2']
    lex = pd.DataFrame([
        [0.0, 1.0],
        [1.0, 1.0]], index=msgs, columns=states)

    print("="*70 + "\nEven priors and all-0 message costs\n")
    basic_mod = RSA(lexicon=lex, prior=[0.5, 0.5], costs=[0.0, 0.0])

    display_reference_game(basic_mod)

    print("\nLiteral listener")
    display(basic_mod.literal_listener())

    print("\nPragmatic speaker")
    display(basic_mod.speaker())

    print("\nPragmatic listener")
    display(basic_mod.listener())

Even priors and all-0 message costs



,r1,r2,costs
hat,0.0,1.0,0.0
glasses,1.0,1.0,0.0
prior,0.5,0.5,
alpha,1.0,,



Literal listener


,r1,r2
hat,0.0,1.0
glasses,0.5,0.5



Pragmatic speaker


,hat,glasses
r1,0.000000,1.000000
r2,0.666667,0.333333



Pragmatic listener


,r1,r2
hat,0.00,1.00
glasses,0.75,0.25


### 2. Референциальная игра:

**Три объекта (r1, r2, r3):**

|   |  r1  |  r2  |  r3 |
| -- | -- | -- | -- |
| шляпа | 1 | 1 | 0 |
|  очки  |  0  |  0  | 1 |
|  усы  |  0  |  1  |  0 |


**Исходная вероятность (P(r), prior)**

| |P(r)  |
| -- | -- |
|r1|	1/3|
|r2|	1/3|
|r3|	1/3|

**Цена высказывания (C(u), cost)**

| | C(u) |
| -- | -- |
|'шляпа'	|0|
|'очки'	|0|
|'усы'	|0|

**Если принять, что alpha = 1, то распределение вероятностей для прагматического слушающего такое:**

|   |  r1  |  r2  |  r3 |
| -- | -- | -- | -- |
| шляпа | 0.75 | 0.25 | 0 |
|  очки  |  0  |  0  | 1 |
|  усы  |  0  |  1  |  0 |

#### 1. можно ли вывести скалярную импликатуру из таблицы прагматического слушающего? обоснуйте свой ответ

#### 2.  что изменится, с точки зрения вычисления импликатуры, если поменять исходные вероятности на P(r1) = 0.1, P(r2) = P(r3) = 0.45? Приведите соответствующую таблицу распределения вероятностей для прагматического слушающего и прокомментируйте её. При расчетах, можно использовать код из пункта 1.

<font color="green"> Ответ: </font>

1. Для вывода скалярной импликатуры из таблицы прагматического слушающего необходимо определить, какие объекты наиболее вероятно имеются ввиду, когда говорят определенные слова и имеют ли они разную вероятность для разных ситуаций (является ли одно сообщение явно предпочтительнее других). 

  Заметим, что *усы* подразумевает только ситуацию r2, в то время как *шляпа* имеет ненулевые вероятности и в r1, и в r2. Следовательно, мы можем вывести импликатуру: "объект r1 - не *усы*"

2. Таблица распределения вероятностей для прагматического слушающего будет выглядеть следующим образом:

| | r1	| r2	|	r3	|
|--|--	|--	|	--	|
| шляпа	| 0.330579	|	0.669421	|	0.0	|
| очки	| 0.000000	|	0.000000	|	1.0	|
| усы	| 0.000000	|	1.000000	|	0.0	|

В исходном случае с равными исходными вероятностями, прагматический слушатель при сообщении *шляпа* присваивает наибольшую вероятность объекту r1, хотя и у объекта r2 она ненулевая, но там есть абсолютная вероятность у *усов*, поэтому r2 - не шляпа.

В данном же случае, при сообщении *шляпа* наиболее вероятным объектом становится r2 и возникает конфликт между *шляпой* и *усами*, что из них более вероятно объект r2? Соответственно, здесь уже нет такой четко выводимой скалярной импликатуры как в 1 случае.

In [4]:
# вычисление новых значений вероятностей
import numpy as np
import pandas as pd


class RSA:
    def __init__(self, lexicon, prior, costs, alpha=1.0):
        self.lexicon = lexicon
        self.prior = np.array(prior)
        self.costs = np.array(costs)
        self.alpha = alpha

    def literal_listener(self):
        return rownorm(self.lexicon * self.prior) # Возвращаем матрицу self.lexicon с учетом self.prior

    def speaker(self):
        lit = self.literal_listener().T
        utilities = self.alpha * (safelog(lit) + self.costs)
        return rownorm(np.exp(utilities))

    def listener(self):
        # Ваш код тут
        speaker_probs = self.speaker().T
        weighted_speaker_probs = speaker_probs * self.prior    # вычисляем ожидаемые значения для каждого сообщения для каждого сообщения, учитывая lexicon, prior, costs и alpha
        return rownorm(weighted_speaker_probs) # Ваш код тут


def rownorm(mat):    # для нормализации строк матрицы
    """Row normalization of np.array or pd.DataFrame"""
    return (mat.T / mat.sum(axis=1)).T


def safelog(vals):
    """Silence distracting warnings about log(0)."""
    with np.errstate(divide='ignore'):
        return np.log(vals)


if __name__ == '__main__':
    """Example from the class slides"""

    from IPython.display import display


    def display_reference_game(mod):
        d = mod.lexicon.copy()
        d['costs'] = mod.costs
        d.loc['prior'] = list(mod.prior) + [""]
        d.loc['alpha'] = [mod.alpha] + [" "] * mod.lexicon.shape[1]
        display(d)


    # Core lexicon:
    msgs = ['шляпа', 'очки', 'усы']
    states = ['r1', 'r2', 'r3']
    lex = pd.DataFrame([
        [1, 1, 0],
        [0, 0, 1],
        [0, 1, 0]], index=msgs, columns=states)
    
    prior = [0.1, 0.45, 0.45]
    costs = [0, 0, 0]
        
    modified_mod = RSA(lexicon=lex, prior=prior, costs=costs)

    print("\nLiteral listener")
    display(modified_mod.literal_listener())

    print("\nPragmatic speaker")
    display(modified_mod.speaker())

    print("\nPragmatic listener")
    display(modified_mod.listener())


Literal listener


,r1,r2,r3
шляпа,0.181818,0.818182,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0



Pragmatic speaker


,шляпа,очки,усы
r1,1.00,0.0,0.00
r2,0.45,0.0,0.55
r3,0.00,1.0,0.00



Pragmatic listener


,r1,r2,r3
шляпа,0.330579,0.669421,0.0
очки,0.000000,0.000000,1.0
усы,0.000000,1.000000,0.0


 ### 3. Рассмотрите референциальные игры с тремя объектами и с тремя высказываниями, где можно вычислить две скалярные импликатуры:
- приведите несколько вариантов таких игр (можно использовать те, которые были в эксперименте в конце занятия 16 мая)

- последовательно проанализируйте, как варьирование параметров цены высказывания, исходной вероятности выбора объекта, оптимальности альфа влияет на вычисление скалярных импликатур в таких играх

При выполнении этого пункта можно использовать код из пункта 1.

<font color="green"> Ответ: </font>

<font color="green">**1 игра.** </font>

Например, у нас есть 3 объекта:
- первый имеет бороду и очки
- второй имеет очки и галстук
- третий имеет только галстук

S0: literal speaker

| | beard | glasses | tie|
|--|--|--|--|
|r1 | 0.5 | 0.5 | 0|
|r2 | 0 | 0.5 | 0.5|
|r3 | 0 | 0 | 1 |

L1: pragmatic listener

| | r1 | r2 | r3|
|--|--|--|--|
|beard | 1 | 0 | 0|
|glasses | 0.4 | 0.6 | 0|
|tie | 0 | 0.33 | 0.67|

В данной игре есть две скалярные импликатуры:

1. r2 - не галстук, так как галстук - r3
2. r1 - не очки, так как r1 - борода, а r2 - очки

In [16]:
# изменим второй из параметров, отвечающих за цену высказывания
msgs = ['beard', 'glasses', 'tie']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.5, 0, 0],
    [0.5, 0.5, 0],
    [0, 0.5, 0.5]], index=msgs, columns=states)

prior = [1/3, 1/3, 1/3]
costs = [0, 8, 0]

modified_mod = RSA(lexicon=lex, prior=prior, costs=costs)

print("\nLiteral listener")
display(modified_mod.literal_listener())

print("\nPragmatic speaker")
display(modified_mod.speaker())

print("\nPragmatic listener")
display(modified_mod.listener())


Literal listener


,r1,r2,r3
beard,1.0,0.0,0.0
glasses,0.5,0.5,0.0
tie,0.0,0.5,0.5



Pragmatic speaker


,beard,glasses,tie
r1,0.00067,0.999330,0.000000
r2,0.00000,0.999665,0.000335
r3,0.00000,0.000000,1.000000



Pragmatic listener


,r1,r2,r3
beard,1.000000,0.000000,0.000000
glasses,0.499916,0.500084,0.000000
tie,0.000000,0.000335,0.999665


**в целом, почти ничего не изменилось**

In [18]:
# изменим третий из параметров, отвечающих за цену высказывания
msgs = ['beard', 'glasses', 'tie']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.5, 0, 0],
    [0.5, 0.5, 0],
    [0, 0.5, 0.5]], index=msgs, columns=states)

prior = [1/3, 1/3, 1/3]
costs = [0, 0, 8]

modified_mod = RSA(lexicon=lex, prior=prior, costs=costs)

print("\nLiteral listener")
display(modified_mod.literal_listener())

print("\nPragmatic speaker")
display(modified_mod.speaker())

print("\nPragmatic listener")
display(modified_mod.listener())


Literal listener


,r1,r2,r3
beard,1.0,0.0,0.0
glasses,0.5,0.5,0.0
tie,0.0,0.5,0.5



Pragmatic speaker


,beard,glasses,tie
r1,0.666667,0.333333,0.000000
r2,0.000000,0.000335,0.999665
r3,0.000000,0.000000,1.000000



Pragmatic listener


,r1,r2,r3
beard,1.000000,0.000000,0.000000
glasses,0.998995,0.001005,0.000000
tie,0.000000,0.499916,0.500084


**теперь уже получилось что-то интересное, борода и очки сражаются за r1, но так как у очков еще есть вероятность на r2, то r2 все-таки очки, а r3 - галстук**

Но здесь прям очень схожие вероятности у галстука на r2 и r3, поэтому я даже не знаю, можно ли назвать это скалярной импликатурой

In [19]:
# изменим исходные вероятности
msgs = ['beard', 'glasses', 'tie']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.5, 0, 0],
    [0.5, 0.5, 0],
    [0, 0.5, 0.5]], index=msgs, columns=states)

prior = [0.2, 0.6, 0.2]
costs = [0, 0, 0]

modified_mod = RSA(lexicon=lex, prior=prior, costs=costs)

print("\nLiteral listener")
display(modified_mod.literal_listener())

print("\nPragmatic speaker")
display(modified_mod.speaker())

print("\nPragmatic listener")
display(modified_mod.listener())


Literal listener


,r1,r2,r3
beard,1.00,0.00,0.00
glasses,0.25,0.75,0.00
tie,0.00,0.75,0.25



Pragmatic speaker


,beard,glasses,tie
r1,0.8,0.2,0.0
r2,0.0,0.5,0.5
r3,0.0,0.0,1.0



Pragmatic listener


,r1,r2,r3
beard,1.000000,0.000000,0.0
glasses,0.117647,0.882353,0.0
tie,0.000000,0.600000,0.4


Здесь видна импликатура: так как борода - r1, то r2 - очки, хотя у галстука r2>r3, но получается, что **r2 - не галстук** 

In [22]:
# теперь изменим alpha
msgs = ['beard', 'glasses', 'tie']
states = ['r1', 'r2', 'r3']
lex = pd.DataFrame([
    [0.5, 0, 0],
    [0.5, 0.5, 0],
    [0, 0.5, 0.5]], index=msgs, columns=states)

prior = [1/3, 1/3, 1/3]
costs = [0, 0, 0]

modified_mod = RSA(lexicon=lex, prior=prior, costs=costs, alpha = 10)

print("\nLiteral listener")
display(modified_mod.literal_listener())

print("\nPragmatic speaker")
display(modified_mod.speaker())

print("\nPragmatic listener")
display(modified_mod.listener())


Literal listener


,r1,r2,r3
beard,1.0,0.0,0.0
glasses,0.5,0.5,0.0
tie,0.0,0.5,0.5



Pragmatic speaker


,beard,glasses,tie
r1,0.999024,0.000976,0.0
r2,0.000000,0.500000,0.5
r3,0.000000,0.000000,1.0



Pragmatic listener


,r1,r2,r3
beard,1.000000,0.000000,0.000000
glasses,0.001947,0.998053,0.000000
tie,0.000000,0.333333,0.666667


**Снова повторяется импликатура, что r2 - не галстук и это видно очень четко, так как и максимальные вероятности все на своих местах**

<font color="green"> **2. Другая игра: фигуры** </font>

- синий квадрат
- синий круг
- зеленый квадрат

Скаларняые импликатуры:
- зелёный -> зеленый квадрат
- круг -> синий круг

L0: literal listener

| | blue-square | blue-circle | green-square|
|--|--|--|--|
|blue | 0.5 | 0.5 | 0 |
|green | 0 | 0 | 1 |
|square | 0.5 | 0 | 0.5 |
| circle | 0 | 1 | 0 |

S1: pragmatic speaker

|  | blue | green | square | circle |
|--|--|--|--|--|
| blue-square | 0.5 | 0 | 0.5 | 0 |
| blue-cicle | 0.33 | 0 | 0 | 0.66 |
| green-square| 0 | 0.66 | 0.33 | 0 |

L1: pragmatic listener

| | blue-square | blue-circle | green-square|
|--|--|--|--|
|blue | 0.6 | 0.4 | 0 |
|green | 0 | 0 | 1 |
|square | 0.6 | 0 | 0.4 |
| circle | 0 | 1 | 0 |

### Предложите вариант применения теории рационального речевого акта к лингвистическому феномену, который не обсуждался нами на занятиях. 

Поясните, почему Вы считаете, что теория рационального речевого акта позволит взглянуть на этот феномен с интересной точки зрения; нужно ли будет модифицировать формулы, как и почему; какую роль в формулах будут играть параметры (цена высказывания, исходная вероятность, оптимальность альфа или какие-то другие параметры, предложенные Вами).

<font color="green">Ответ:</font>
Можно попробовать анализировать разницу между тем, что говорится прямо (эксплицитно) и тем, что подразумевается (имплицитно) в рамках коммуникации. Например, если кто-то говорит: "Здесь жарко", это может быть имплицитным приглашением закрыть окно.